In [2]:
import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text

import pandas as pd
import numpy as np


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error

from official.nlp import optimization

import matplotlib.pyplot as plt

In [3]:
path =  "E:/LOMBA/Kaggle/Comp/Coommon readilbility/"
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')


In [4]:
train.head()
text = train.excerpt.to_numpy()
target = train.target.to_numpy()

In [5]:
def create_folds(data, target="target", num_splits = 5): 
    data["kfold"] = -1 
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Applying Sturg's rule to calculate the no. of bins for target
    num_bins = int(1 + np.log2(len(data))) 

    data.loc[:, "bins"] = pd.cut(data[target], bins=num_bins, labels=False) 
    
    kf = StratifiedKFold(n_splits=num_splits)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)): 
        data.loc[v_, 'kfold'] = f
        
    data = data.drop(["bins"], axis = 1)         
    return data 

In [7]:
cv_data_df = create_folds(train, target = 'target', num_splits = 5)
cv_data_df.kfold.value_counts()

0    567
2    567
1    567
3    567
4    566
Name: kfold, dtype: int64

In [4]:
train_x, test_x, train_y, test_y = train_test_split(txt_clean, target, test_size=0.2, shuffle=True, random_state=42)
np.array(train_x).shape

(2267,)

In [8]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "http://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/albert_en_base/3",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]

In [9]:
tf.keras.layers.reshape((-1, 768))

AttributeError: module 'tensorflow.keras.layers' has no attribute 'reshape'

In [43]:
def albert_base():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]   
    seq_output = outputs["sequence_output"]  
    pooled_output = tf.keras.layers.reshape((-1, 768))(pooled_output)
    net = tf.keras.layers.Dropout(0.1)(pooled_output)
    net = tf.keras.layers.Dense(10, activation="relu")(net)
    net = tf.keras.layers.Dense(1, name='regressor')(net)
    model = tf.keras.models.Model(inputs=text_input, outputs=net, name="large_albert")
    return model

In [44]:
large_albert = albert_large_model()
large_albert.summary()

Model: "large_albert"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      {'input_mask': (None 0           input_10[0][0]                   
__________________________________________________________________________________________________
keras_layer_5 (KerasLayer)      {'default': (None, 7 11683584    keras_layer_4[1][0]              
                                                                 keras_layer_4[1][1]              
                                                                 keras_layer_4[1][2]              
_______________________________________________________________________________________

In [38]:
epochs = 3
batch_size = 32
steps_per_epoch = round(1889//batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [26]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_root_mean_squared_error', min_delta=0.0001, patience=0, verbose=0,
    mode='min')

In [27]:
large_albert.compile(optimizer=optimizer,
                         loss="mean_squared_error",
                         metrics=tf.keras.metrics.RootMeanSquaredError())

In [45]:
str_score = 5
for fold in range(3):
    print (fold)
    train_df = cv_data_df.loc[cv_data_df.kfold != fold].reset_index(drop = True)
    val_df = cv_data_df.loc[cv_data_df.kfold == fold].reset_index(drop = True)
    print (train_df.shape, val_df.shape)
    
    text = train_df.clean_txt
    target = train_df.target
    
    text_val = val_df.clean_txt
    target_val = val_df.target
    
    large_albert = albert_large_model()
    
    epochs = 2
    batch_size = 32
    steps_per_epoch = round(1889//batch_size)
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')
    
    large_albert.compile(optimizer=optimizer,
                         loss="mean_squared_error",
                         metrics=tf.keras.metrics.RootMeanSquaredError())
    large_albert.fit(x=np.array(text),
                y=target,
                validation_data=(np.array(text_val), target_val),
                epochs=epochs,
                batch_size= batch_size)
    values_score = large_albert.evaluate(np.array(text_val), target_val)[1]
    print (values_score)
    if values_score < str_score:
        large_albert.save("saved_model/4")
        str_score = values_score
        print ("Best Iteration ", fold, " With RMSE ", values_score)

0
(1889, 8) (945, 8)
Epoch 1/2
60/60 [==============================] - 940s 16s/step - loss: 0.6148 - root_mean_squared_error: 0.7841 - val_loss: 0.5068 - val_root_mean_squared_error: 0.7119
Epoch 2/2
30/30 [==============================] - 133s 4s/step - loss: 0.3529 - root_mean_squared_error: 0.5940
0.5940473675727844


INFO:tensorflow:Assets written to: saved_model/4\assets


INFO:tensorflow:Assets written to: saved_model/4\assets


Best Iteration With RMSE 0.5940473675727844
1
(1889, 8) (945, 8)
Epoch 1/2
60/60 [==============================] - 935s 16s/step - loss: 0.6224 - root_mean_squared_error: 0.7889 - val_loss: 0.3739 - val_root_mean_squared_error: 0.6115
Epoch 2/2
30/30 [==============================] - 139s 5s/step - loss: 0.3289 - root_mean_squared_error: 0.5735
0.5735024809837341


INFO:tensorflow:Assets written to: saved_model/4\assets


INFO:tensorflow:Assets written to: saved_model/4\assets


Best Iteration With RMSE 0.5735024809837341
2
(1890, 8) (944, 8)
Epoch 1/2
60/60 [==============================] - 1016s 17s/step - loss: 0.5108 - root_mean_squared_error: 0.7147 - val_loss: 0.3955 - val_root_mean_squared_error: 0.6289
Epoch 2/2
30/30 [==============================] - 139s 5s/step - loss: 0.2530 - root_mean_squared_error: 0.5030
0.5029637217521667


INFO:tensorflow:Assets written to: saved_model/4\assets


INFO:tensorflow:Assets written to: saved_model/4\assets


Best Iteration With RMSE 0.5029637217521667


In [46]:
import tensorflow.keras.backend as K
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 
exp_albert = albert_large_model()
epochs = 3
batch_size = 32
steps_per_epoch = round(1889//batch_size)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

exp_albert.compile(optimizer=optimizer,
                         loss=root_mean_squared_error,
                         metrics=tf.keras.metrics.RootMeanSquaredError())
exp_albert.summary()

Model: "large_albert"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      {'input_mask': (None 0           input_14[0][0]                   
__________________________________________________________________________________________________
keras_layer_5 (KerasLayer)      {'default': (None, 7 11683584    keras_layer_4[5][0]              
                                                                 keras_layer_4[5][1]              
                                                                 keras_layer_4[5][2]              
_______________________________________________________________________________________

In [48]:
exp_albert.fit(x=np.array(text),
                y=target,
                validation_data=(np.array(text_val), target_val),
                epochs=epochs,
                batch_size= batch_size)

Epoch 1/3
60/60 [==============================] - 1010s 17s/step - loss: 0.6286 - root_mean_squared_error: 0.6468 - val_loss: 0.6981 - val_root_mean_squared_error: 0.7029
Epoch 2/3
60/60 [==============================] - 1019s 17s/step - loss: 0.4852 - root_mean_squared_error: 0.4908 - val_loss: 0.5069 - val_root_mean_squared_error: 0.5137
Epoch 3/3
60/60 [==============================] - 1038s 17s/step - loss: 0.4003 - root_mean_squared_error: 0.4042 - val_loss: 0.5020 - val_root_mean_squared_error: 0.5082


In [50]:
exp_albert.save("saved_model/5")

INFO:tensorflow:Assets written to: saved_model/5\assets


INFO:tensorflow:Assets written to: saved_model/5\assets


In [49]:
exp_albert.evaluate(np.array(text_val), target_val)

30/30 [==============================] - 143s 5s/step - loss: 0.5020 - root_mean_squared_error: 0.5082


[0.5020362138748169, 0.5082021355628967]

In [39]:
tf.__version__

'2.5.0'

In [43]:
tf.keras.models.load_model(
    "saved_model/3", custom_objects={"AdamWeightDecay": optimizer}
)

In [47]:
tf.keras.models.load_model(
   "saved_model/albert_base.h5")

TypeError: load_model() got an unexpected keyword argument 'save_traces'